In [18]:
!pip3 install seaborn


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/tokenizers-0.14.1-py3.11-macosx-12-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/urllib3-1.26.17-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/sympy-1.12-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/google_auth-2.23.3-py3.11.egg is deprecated. p

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 2.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler,LabelEncoder
from sklearn.decomposition import PCA
import sqlite3
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
import datetime as dt
from sklearn import linear_model
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
import json
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import openai
import pandas as pd
import sqlite3
from openai.api_resources.completion import Completion

In [98]:
# Connect to the sqlite db file and retrieve data as Pandas data frame.
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
sql = "select * from fires"
df = pd.read_sql_query(sql, cnx)

In [99]:
df.head()
str(df.columns)

"Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape'], dtype='object')"

In [108]:
df_interest = df[["FIRE_YEAR","DISCOVERY_DOY","DISCOVERY_TIME","CONT_DOY","CONT_TIME","STAT_CAUSE_CODE","STAT_CAUSE_DESCR","FIRE_SIZE","LATITUDE","LONGITUDE","STATE","COUNTY","FIPS_NAME","DISCOVERY_DATE"]]
df_interest.isna().sum()

FIRE_YEAR                0
DISCOVERY_DOY            0
DISCOVERY_TIME      882638
CONT_DOY            891531
CONT_TIME           972173
STAT_CAUSE_CODE          0
STAT_CAUSE_DESCR         0
FIRE_SIZE                0
LATITUDE                 0
LONGITUDE                0
STATE                    0
COUNTY              678148
FIPS_NAME           678148
DISCOVERY_DATE           0
dtype: int64

In [109]:
print(f"Number of removed duplicate rows: {df_interest.duplicated().sum()}")
df_interest = df_interest.drop_duplicates()

Number of removed duplicate rows: 3525


In [110]:
df_interest["combined_date_dis"] = df_interest["FIRE_YEAR"]*1000 + df_interest["DISCOVERY_DOY"]
df_interest["combined_date_dis"] = pd.to_datetime(df_interest["combined_date_dis"], format = "%Y%j")
df_interest = df_interest.set_index("combined_date_dis")
df_interest.head()

,FIRE_YEAR,DISCOVERY_DOY,DISCOVERY_TIME,CONT_DOY,CONT_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_NAME,DISCOVERY_DATE
combined_date_dis,,,,,,,,,,,,,,
2005-02-02,2005,33,1300,33.0,1730,9.0,Miscellaneous,0.10,40.036944,-121.005833,CA,63,Plumas,2453403.5
2004-05-12,2004,133,0845,133.0,1530,1.0,Lightning,0.25,38.933056,-120.404444,CA,61,Placer,2453137.5
2004-05-31,2004,152,1921,152.0,2024,5.0,Debris Burning,0.10,38.984167,-120.735556,CA,17,El Dorado,2453156.5
2004-06-28,2004,180,1600,185.0,1400,1.0,Lightning,0.10,38.559167,-119.913333,CA,3,Alpine,2453184.5
2004-06-28,2004,180,1600,185.0,1200,1.0,Lightning,0.10,38.559167,-119.933056,CA,3,Alpine,2453184.5


In [100]:


# Set up OpenAI API key
openai.api_key = 'xxxxxxxxxxxxx'



In [50]:
# Connect to the SQLite database
# conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
# cursor = conn.cursor()
# sql = "select * from fires"
# df = pd.read_sql_query(sql, conn)

In [113]:


def query_database(sql_query):
    """
    Executes a SQL query against the SQLite database.
    """
    return pd.read_sql_query(sql_query, cnx)

def generate_sql_query(messages):
    """
    Uses OpenAI's GPT-4 to generate a SQL query based on a natural language prompt.
    """
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )
    return response.choices[0].message['content'].strip()


def run_tool(json_query):
    """
    Uses OpenAI's GPT-4 to generate a SQL query based on a natural language prompt.
    """
    if(json_query["tool_name"] == "Query_Database"):
        try:
            df1 = query_database(json_query["command"])
            print(df1)
            data = df1.head().to_dict(orient='records')  # Converts the DataFrame head to a list of dictionaries
            return data
        except:
            print("Incorrect sql command. Trying again")
            return "Retry SQL Query"
    if(json_query["tool_name"] == "Generate_Final_Answer"):
        return "Finish"
        

In [128]:



columns = str(df.columns)
memory = [columns]
prompt = '''<s> [INST]You are an agent capable of using a variety of TOOLS to answer a data analytics question.
Always use MEMORY to help select the TOOLS to be used. The name of the SQLlite table is fires.
 
MEMORY
The column names are 
'''
prompt+= columns
prompt+= '''
 
TOOLS
- Generate_Final_Answer: Use if answer to User's question can be given with MEMORY and data we have. Use this after thorough data analysis
- Query_Database: Write only one SQL Query to query the Database. This can only execute one query at a time
 
ANSWER FORMAT
{
    "tool_name": """
    "command": """
}
[/INST]
User:  Have wildfires become more or less frequent over time?
Assistant: {
 "tool_name":
 "command":
}
''' 
messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
]
# Execute the generated SQL query
sql_query = generate_sql_query(messages)
print(sql_query)
print(json.loads(sql_query))
json_query = json.loads(sql_query)

data = run_tool(json_query)

prompt+= sql_query+ "\n ''response:''" + str(data) + "}\n"

messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
]
# messages.append({"role":"system", "content": "SQL data extracted"})
# messages.append({"role":"user", "content": "Move to the next step as specified in the prompt"})
generate_sql_query(messages)




{
    "tool_name": "Query_Database",
    "command": "SELECT FIRE_YEAR, COUNT(*) AS NUM_WILDFIRES FROM fires GROUP BY FIRE_YEAR ORDER BY FIRE_YEAR;"
}
{'tool_name': 'Query_Database', 'command': 'SELECT FIRE_YEAR, COUNT(*) AS NUM_WILDFIRES FROM fires GROUP BY FIRE_YEAR ORDER BY FIRE_YEAR;'}
    FIRE_YEAR  NUM_WILDFIRES
0        1992          67975
1        1993          61989
2        1994          75955
3        1995          71472
4        1996          75574
5        1997          61450
6        1998          68370
7        1999          89363
8        2000          96416
9        2001          86587
10       2002          75656
11       2003          68261
12       2004          69279
13       2005          88604
14       2006         114004
15       2007          95573
16       2008          85378
17       2009          78325
18       2010          79889
19       2011          90552
20       2012          72769
21       2013          64780
22       2014          67753
23       2015 

'{\n    "tool_name": "Generate_Final_Answer",\n    "command": "Based on the data retrieved from the \'fires\' table, the number of wildfires has fluctuated over the years. For example, in 1992 there were 67,975 wildfires, while in 1994 there were 75,955 wildfires. This suggests that the frequency of wildfires has varied over time, and further analysis may be needed to determine any overall trend."\n}'

In [76]:
# sql_query = '''SELECT COUNTY, COUNT(*) AS fire_count FROM fires GROUP BY COUNTY ORDER BY fire_count DESC LIMIT 1\nUNION\nSELECT COUNTY, COUNT(*) AS fire_count FROM fires GROUP BY COUNTY ORDER BY fire_count ASC LIMIT 1'''
# # df1 = query_database(sql_query)
# # print(df1)
# print(json.load(sql_query))


AttributeError: 'str' object has no attribute 'read'